# Running Canonical Monte Carlo Simulated Annealing on an Ewald Hamiltonian
When generating a set of structures to sample, it may be useful to enumerate them through a Monte Carlo (MC) simulation. A MC with only Ewald electrostatic energies can be especially useful for the initial training set, when there are no DFT energies to parametrize a cluster expansion and run MC. 

In [ ]:
import copy

import numpy as np
import matplotlib.pyplot as plt
from monty.serialization import loadfn, dumpfn
from pymatgen.analysis.ewald import EwaldSummation
from pymatgen.core import Structure

from smol.cofe import ClusterSubspace
from smol.cofe.extern import EwaldTerm

### (0) Create a Cluster Subspace based on the disordered structure with only the empty cluster and Ewald term

In [ ]:
lno = loadfn('data/lno_prim.json') 
lno

In [ ]:
empty_cutoff = {} # Defining the cut-offs as an empty dictionary will generate a subspace with only the empty cluster

subspace = ClusterSubspace.from_cutoffs(structure=lno, 
                                        cutoffs=empty_cutoff, 
                                        basis='sinusoid',
                                        supercell_size='O2-')

subspace.add_external_term(EwaldTerm(eta=None)) # Add the external Ewald Term

### (1) Create an Ewald Processor
An Ewald Processor handles changes in electrostatic interaction energies among different configurational states, using an Ewald Summation term.

In [ ]:
from smol.moca import EwaldProcessor

In [ ]:
# The supercell with which we will run MC on
sc_matrix = np.array([[4, 0, 0],
                      [0, 4, 0],
                      [0, 0, 2]])

# Specifying the dielectric constant, the inverse of which is parametrized when fitting a CE with electrostatics (Example 1-1). 
dielectric = 5 

# Creating the Ewald Processor
ewald_proc = EwaldProcessor(cluster_subspace=subspace,
                            supercell_matrix=sc_matrix,
                            ewald_term=EwaldTerm(),
                            coefficient=1/dielectric)

### (2) Create a Canonical Ensemble

In [ ]:
from smol.moca import CanonicalEnsemble

# Create the canonical ensemble directly from the Ewald Processor, without creating a Cluster Expansion.
ensemble = CanonicalEnsemble(processor=ewald_proc)

# If the goal is to enumerate new structures for DFT calculations, it may be wise to limit the size of
# your supercell such that a relaxation calculation is feasible.
# The thermodynamics may not be the most realistic, but you can generate training structures
# that have relatively low electrostatic energies, which may translate to lower DFT energies.
print(f'The supercell size for the processor is {ensemble.processor.size} prims.')
print(f'The ensemble has a total of {ensemble.num_sites} sites.')
print(f'The active sublattices are:')
for sublattice in ensemble.sublattices:
    print(sublattice)

### (3) Create the Sampler

In [ ]:
from smol.moca import Sampler

sampler = Sampler.from_ensemble(ensemble, temperature=10000)
print(f"Sampling information: {sampler.samples.metadata}")

### (4) Generate an initial ordered structure to start the MC simulation.

In [ ]:
from pymatgen.transformations.standard_transformations import OrderDisorderedStructureTransformation

# Here we will just use the order disordered transformation from
# pymatgen to get an ordered version of a prim supercell.
# The structure will have the same composition set in the prim.
transformation = OrderDisorderedStructureTransformation(algo=2)

supercell = lno.copy()
supercell.make_supercell(sc_matrix)

test_struct = transformation.apply_transformation(supercell)
print(test_struct.composition)

In [ ]:
# Obtain the initial occupancy string from the
# test structure created above.
init_occu = ensemble.processor.occupancy_from_structure(test_struct)

# The occupancy strings created by the processor
# are by default "encoded" by the indices of the species
# for each given site. You can always see the actual
# species in the occupancy string by decoding it.
print(f'The encoded occupancy is:\n{init_occu}')
print(f'The initial occupancy is:\n {ensemble.processor.decode_occupancy(init_occu)}')

### (5) Run MC simulated annealing

In [ ]:
# Setting up the range of temperatures for simulated annealing. We start at very 
# high temperatures to approach the random limit. At each temperature, a MC simulation is performed. 
# At the lowest temperatures, you may find that you converge to a ground state.
temps = np.logspace(4, 2, 10)

mc_steps = 100000 # Defining number of MC steps at each temperature
n_thin_by = 10 # Number to thin by

# Start simulated annealing.
sampler.anneal(temperatures=temps, 
               mcmc_steps=mc_steps,
               initial_occupancies=init_occu,
               thin_by=n_thin_by, # Saving every 10 samples
               progress=True # Show the progress bar to know how far along you are
              )

### (6) Analyzing MC sampling at each temperature

In [ ]:
n = int(mc_steps/10) # number of samples saved for the MC at each temperature
energies = sampler.samples.get_energies()
mc_temps = list() # Create list of temperatures that correspond to the energies

for t in temps:
    mc_temps.extend([t for i in range(n)]) 

In [ ]:
# Obtain the average and standard deviation of energy at each temperature.
for t in temps:
    plot_inds = np.where(mc_temps == t)[0]
    energies_t = np.array([energies[ind] for ind in plot_inds]) / ewald_proc.size
    avg_en = round(np.average(energies_t), 3)
    std_en = round(np.std(energies_t), 4)
    print(f'At T = {round(t, 2)} K \nAverage energy = {avg_en} eV/prim \nStd dev = {std_en} eV/prim \n')

#### Obtaining the ground state

In [ ]:
lowest_en = sampler.samples.get_minimum_energy() / ewald_proc.size
lowest_en_occu = sampler.samples.get_minimum_energy_occupancy()

print(f'The ground state energy is {lowest_en} eV/prim')
print(f'Ground state occupancy is {lowest_en_occu}')

### (7) Plotting the results

In [ ]:
x_ind = 0 # To keep track of x axis indexing

for i, t in enumerate(temps):
    plot_inds = np.where(mc_temps == t)[0]
    energies_t = np.array([energies[ind] for ind in plot_inds]) / ewald_proc.size # Obtain normalized energies at this temperature
    col = plt.cm.plasma(i/len(temps)) # Set the color
    num_samples = len(energies_t) 
    plt.plot(np.arange(x_ind, x_ind + num_samples), energies_t, color=col,   
             label=f'{round(t, 2)} K')
    x_ind += num_samples
    
plt.legend()
plt.title('Simulated annealing on an \n Ewald Hamiltonian', fontsize=16)
plt.ylabel('Ewald Energy (eV/prim)', fontsize=14)
plt.xlabel('MC samples', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

### (8) Save the samples

In [ ]:
from smol.io import save_work

save_work('data/lno_ewald_sim_anneal_samples.mson', sampler.samples)